In [1]:
from openai import AzureOpenAI
import json
import os
import Functions
from Functions import get_weather_for_city, get_restaurants_for_city,get_hotels_for_city
import time

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

# Create assistant
assistant = client.beta.assistants.create(
  name="Travel Assistant",
  instructions="""You are an assistant that will provide an itinerary for a trip to a city, list of restaurants, and weather of that city  . 
  """,
  model=azure_deployment, 
  tools=Functions.tools_function)

# Create thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_lMSmhGqSQ44fbarQ56OaUAlR', created_at=1729059747, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [2]:
prompt_prefix = """
  You will be given a city name and you should return the following: 
  1. a list of activities to do in that city.
  2. a list of hotels to stay at in that city
  3. a list of restaurants to eat at in that city.
  4. the current weather in that city.
  """

user_question ="""What can I do in Manila?
"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=prompt_prefix + user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    # the assistant requires calling some functions and submit the tool outputs back to the run
    available_functions = {
      "get_weather_for_city": get_weather_for_city,
      "get_restaurants_for_city": get_restaurants_for_city,
      "get_hotels_for_city": get_hotels_for_city
    }
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
          
      if function_name == "get_weather_for_city":
        function_response = get_weather_for_city(city=function_args.get("city"))
      elif function_name == "get_restaurants_for_city":
        function_response = get_restaurants_for_city(city=function_args.get("city"))
      elif function_name == "get_hotels_for_city":
        function_response = get_hotels_for_city(city=function_args.get("city"))    
      else:
        function_response = "Unknown function"

      tool_outputs.append(
        { 
          "tool_call_id": tool_call.id,
          "output": str(function_response)
        }
      )
      run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
      )
  
    else:
      print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

queued
queued
queued
Here are some details for your trip to Manila:

Activities in Manila:
1. Visit the historic Rizal Park (Luneta) and witness the iconic Rizal Monument.
2. Dive into history at Intramuros, the Walled City, which is home to ancient Spanish-era influences.
3. Explore Manila Ocean Park and interact with various marine life in the oceanarium.
4. Check out the National Museum of the Philippines, which houses some of the country's important artifacts.
5. Visit the Manila Baywalk, especially during sunset for the beautiful view.

Hotels in Manila:
1. Okada Manila
![Okada Manila](https://cf.bstatic.com/xdata/images/hotel/150x150/594165114.jpg?k=db63a567f3b4483f9e017031fa7499324c76748f9f289730d00b19da88304e2d&o=)
[More Info](https://www.booking.com/hotel/ph/okada-manila.html)
2. Forbes Area BGC Taguig Manila
![Forbes Area BGC Taguig Manila](https://cf.bstatic.com/xdata/images/hotel/150x150/182510632.jpg?k=e67c844cef07db84497a5941b7a7f545dab56496da9f1e5beb96d60cd794e6d8&o=)
[M